    Ben Christensen
    Fiscal Responsibility Index
    December 4, 2018
    Math 402

In [36]:
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from statsmodels.regression.linear_model import OLS
from itertools import combinations
import numpy as np
from statsmodels.formula.api import Logit

In [96]:
#Import FRI dataset
df = pd.read_csv("scores_w_regions")
#Rename columns so they look pretty in output
df["Year of Birth"] = df["YOB"]
df["Position"].replace("Rep", "Representative", inplace=True)
df["Position"].replace("Sen", "Senator", inplace=True)
df["Party"].replace("R", "Republican", inplace=True)
df["Party"].replace("I", "Independent", inplace=True)
df["Party"].replace("D", "Democrat", inplace=True)
df["FRI"] = df["Score"]
df.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)
#Show summary statistics for Fiscal Responsibility Index
df[["FRI"]].describe()

FRI
count  1.305000e+03
mean  -4.971455e+11
std    6.277852e+11
min   -2.813974e+12
25%   -9.092899e+11
50%   -2.284700e+11
75%   -1.100000e+07
max    3.700000e+09

In [97]:
#Create a pivot table for Party and Position
df.pivot_table(values="FRI", index="Party", columns="Position", aggfunc='mean')

Position     Representative       Senator
Party                                    
Democrat      -3.716170e+11 -1.086663e+12
Independent   -8.597464e+11 -8.531340e+11
Republican    -4.168509e+11 -8.500200e+11

In [98]:
#Find the Number of observation for every level of tenure
for i in range(12):
    print(i,': ',df[df["Tenure"]==i]["Tenure"].count(), sep='')

0: 0
1: 209
2: 171
3: 219
4: 140
5: 111
6: 119
7: 95
8: 71
9: 52
10: 26
11: 92


In [99]:
#One-hot encoding for categorical variables
#df["Democrat"] = 1*(df["Party"] == "Democrat")
df["Republican"] = 1*(df["Party"] == "Republican")
df["Senator"] = 1*(df["Position"]=="Senator")
df = pd.get_dummies(df, drop_first=True, columns=["Region", "Division"])
#Add a tenure squared variable
df["Tenure_sq"] = df["Tenure"] ** 2
df.head()

Name       Party        Position         Score State  \
0     Abercrombie, Neil    Democrat  Representative -1.047578e+12    HI   
1  Mink, Patsy Takemoto    Democrat  Representative -3.926870e+11    HI   
2              Case, Ed    Democrat  Representative -6.337500e+11    HI   
3         Hirono, Mazie    Democrat  Representative -4.689300e+10    HI   
4         Djou, Charles  Republican  Representative -3.000000e+07    HI   

   Tenure   YOB     Avg_score State Code  Year of Birth    ...      \
0       7  1938 -1.496541e+11         HI           1938    ...       
1       3  1927 -1.308957e+11         HI           1927    ...       
2       3  1952 -2.112500e+11         HI           1952    ...       
3       3  1947 -1.563100e+10         HI           1947    ...       
4       1  1970 -3.000000e+07         HI           1970    ...       

   Region_West  Division_East South Central  Division_Middle Atlantic  \
0            1                            0                         0   
1            1                            0                         0   
2            1                            0                         0   
3            1                            0                         0   
4            1                            0                         0   

   Division_Mountain  Division_New England  Division_Pacific  \
0                  0                     0                 1   
1                  0                     0                 1   
2                  0                     0                 1   
3                  0                     0                 1   
4                  0                     0                 1   

   Division_South Atlantic  Division_West North Central  \
0                        0                            0   
1                        0                            0   
2                        0                            0   
3                        0                            0   
4                        0                            0   

   Division_West South Central  Tenure_sq  
0                            0         49  
1                            0          9  
2                            0          9  
3                            0          9  
4                            0          1  

[5 rows x 25 columns]

## (i)

OLS without regularization using all features

In [ ]:
df = df[df["Party"]!="Independent"]

In [108]:
Y = df["Score"]
# I just erased 'Democrat' below
X = df[["Senator", "Republican", "Division_East South Central", "Division_Middle Atlantic", "Division_Mountain", "Division_New England", "Division_Pacific", "Division_South Atlantic", "Division_West North Central", "Division_West South Central", "Tenure", "Tenure_sq", "Year of Birth"]]
X = sm.add_constant(X)
regression = sm.OLS(Y, X).fit()
with open("ols_regression.tex", 'w') as f:
    f.write(regression.summary().as_latex())
print(regression.summary())

                            OLS Regression Results                            
Dep. Variable:                  Score   R-squared:                       0.536
Model:                            OLS   Adj. R-squared:                  0.531
Method:                 Least Squares   F-statistic:                     114.5
Date:                Thu, 13 Dec 2018   Prob (F-statistic):          4.54e-204
Time:                        00:18:14   Log-Likelihood:                -36716.
No. Observations:                1302   AIC:                         7.346e+04
Df Residuals:                    1288   BIC:                         7.353e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [103]:
# I deleted "Democrat" from below
features = np.array(["Senator", "Republican", "Division_East South Central", "Division_Middle Atlantic", "Division_Mountain", "Division_New England", "Division_Pacific", "Division_South Atlantic", "Division_West North Central", "Division_West South Central", "Tenure", "Tenure_sq", "YOB"])
#Prime AIC and BIC with using all features
smallest_AIC = 7.384e+04
smallest_BIC = 7.391e+04
best_aic_subset, best_bic_subset = features, features
for num_features in range(1, len(features)+1):
    for subset in combinations(features, num_features):
        if num_features == 1:
            subset = [subset[0]]
        else:
            subset = list(subset)
        X = df[subset]
        X = sm.add_constant(X)
        Y = df["Score"]
        model = sm.OLS(Y, X).fit()
        if model.aic < smallest_AIC:
            smallest_AIC = model.aic
            best_aic_subset = subset
        if model.bic < smallest_BIC:
            smallest_BIC = model.bic
            best_bic_subset = subset
print("Features that give optimal AIC:")
print(best_aic_subset)
print("AIC:", smallest_AIC)
print("\nFeatures that give optimal BIC:")
print(best_bic_subset)
print("BIC:", smallest_BIC)

Features that give optimal AIC:
['Senator', 'Republican', 'Tenure', 'Tenure_sq', 'YOB']
AIC: 73448.3679794

Features that give optimal BIC:
['Senator', 'Republican', 'Tenure', 'Tenure_sq', 'YOB']
BIC: 73479.3979203


## (ii)

OLS without regularization using features that optimize BIC

In [110]:
X = df[['Senator', 'Republican', 'Tenure', 'Tenure_sq']]#, 'Year of Birth']]
Y = df["Score"]
X = sm.add_constant(X)
regression = sm.OLS(Y, X).fit()
with open("few_regression.tex", 'w') as f:
    f.write(regression.summary().as_latex())
print(regression.summary())

                            OLS Regression Results                            
Dep. Variable:                  Score   R-squared:                       0.511
Model:                            OLS   Adj. R-squared:                  0.509
Method:                 Least Squares   F-statistic:                     338.8
Date:                Thu, 13 Dec 2018   Prob (F-statistic):          1.16e-199
Time:                        00:22:00   Log-Likelihood:                -36750.
No. Observations:                1302   AIC:                         7.351e+04
Df Residuals:                    1297   BIC:                         7.354e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.537e+11   3.92e+10      9.018      0.0

## 4.22

In [105]:
X = df[features]
ols_MSE = -np.mean(cross_val_score(linear_model.LinearRegression(), X,Y,cv=7, scoring="neg_mean_squared_error"))
X = df[['Senator', 'Republican', 'Tenure', 'Tenure_sq', 'YOB']]
few_MSE = -np.mean(cross_val_score(linear_model.LinearRegression(), X,Y,cv=7, scoring="neg_mean_squared_error"))


X = df[features]
ridge_MSE, lasso_MSE = 1e50,1e50
for k in range(-5, 6):
    lmbda = 10**k
    ridge_model = linear_model.Ridge(lmbda)
    rMSE = -np.mean(cross_val_score(ridge_model, X,Y,cv=7, scoring="neg_mean_squared_error"))
    if rMSE < ridge_MSE and rMSE>0:
        ridge_MSE = rMSE
        ridge_coef = ridge_model.fit(X,Y).coef_
        ridge_features = features[ridge_coef!=0]
    lasso_model = linear_model.Lasso(lmbda)
    lMSE = -np.mean(cross_val_score(lasso_model, X,Y,cv=7, scoring="neg_mean_squared_error"))
    if lMSE < lasso_MSE and lMSE>0:
        lasso_MSE = lMSE
        lasso_coef = lasso_model.fit(X,Y).coef_
        lasso_features = features[lasso_coef!=0]

print('(i)')
print("OLS without regularization")
print("Features:", features)
print("MSE:", ols_MSE, '\n')
print('(ii)')
print("OLS with fewer features and without regularization")
print("Features:", ['Position_Senator', 'Republican', 'Tenure', 'Tenure_sq', 'YOB'])
print("MSE:", few_MSE, '\n')
print('(iii)')
print("Ridge Model")
print("Features:", ridge_features)
print("MSE:", ridge_MSE, '\n')
print('(iv)')
print("Lasso Model")
print("Features:", lasso_features)
print("MSE:", lasso_MSE)
    
    

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

(i)
OLS without regularization
Features: ['Senator' 'Republican' 'Division_East South Central'
 'Division_Middle Atlantic' 'Division_Mountain' 'Division_New England'
 'Division_Pacific' 'Division_South Atlantic' 'Division_West North Central'
 'Division_West South Central' 'Tenure' 'Tenure_sq' 'YOB']
MSE: 1.88583422997e+23 

(ii)
OLS with fewer features and without regularization
Features: ['Position_Senator', 'Republican', 'Tenure', 'Tenure_sq', 'YOB']
MSE: 1.86064646662e+23 

(iii)
Ridge Model
Features: ['Senator' 'Republican' 'Division_East South Central'
 'Division_Middle Atlantic' 'Division_Mountain' 'Division_New England'
 'Division_Pacific' 'Division_South Atlantic' 'Division_West North Central'
 'Division_West South Central' 'Tenure' 'Tenure_sq' 'YOB']
MSE: 1.88171887473e+23 

(iv)
Lasso Model
Features: ['Senator' 'Republican' 'Division_East South Central'
 'Division_Middle Atlantic' 'Division_Mountain' 'Division_New England'
 'Division_Pacific' 'Division_South Atlantic' 'Divisi

## Logistic Regression

In [111]:
df["Positive_FRI"] = df["FRI"]>=0
Y = df["Positive_FRI"]
X = df[['Senator', 'Republican', 'Tenure', 'Tenure_sq', "Division_East South Central", "Division_Middle Atlantic", "Division_Mountain", "Division_New England", "Division_Pacific", "Division_South Atlantic", "Division_West North Central", "Division_West South Central"]]
X = sm.add_constant(X)
logit_model = Logit(Y,X).fit()
#with open("logit_regression.tex", 'w') as f:
#    f.write(logit_model.summary().as_latex())
    
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.455624
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           Positive_FRI   No. Observations:                 1302
Model:                          Logit   Df Residuals:                     1289
Method:                           MLE   Df Model:                           12
Date:                Thu, 13 Dec 2018   Pseudo R-squ.:                  0.1128
Time:                        00:22:19   Log-Likelihood:                -593.22
converged:                       True   LL-Null:                       -668.61
                                        LLR p-value:                 3.928e-26
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.2195      0.273      0.803      0.422   